In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [2]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [3]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [4]:
cleandf = [] #note + liste des inputs
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

In [5]:

tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] : #liste arbitraire des actions à enlever
        if input in row[1].keys() :
            row[1].pop(input)

#liste des actions à conserver
all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)

In [6]:
#normalisation v1 (mieux que la v2)

for input in all_inputs :
    tmp = []
    for row in tab :
        if input in row[1].keys() :
            tmp.append(row[1].get(input))
    mean = np.mean(tmp)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/mean

print(tab)



[[5 scroll         5.326207
    mousemove      1.467425
    mouseover      2.060896
    mousedown      4.527188
    blur           1.874312
    click          1.840256
    contextmenu    0.740741
    Name: event, dtype: float64]
 [2 mousemove    0.600310
    scroll       0.611204
    mouseover    0.686965
    mousedown    0.754531
    click        0.920128
    Name: event, dtype: float64]
 [4 mousemove    2.601344
    scroll       0.960464
    mouseover    0.686965
    blur         1.249541
    Name: event, dtype: float64]
 ...
 [3 mousemove    0.533609
    mouseover    0.915954
    mousedown    0.754531
    click        0.920128
    Name: event, dtype: float64]
 [4 mousemove    0.533609
    mouseover    0.457977
    mousedown    0.754531
    click        0.920128
    Name: event, dtype: float64]
 [2 mousemove    0.266805
    mouseover    0.457977
    mousedown    0.754531
    click        0.920128
    Name: event, dtype: float64]]


In [7]:
tmp_all_x = [] #stock temporairement les données d'entrées
tmp_all_y = [] #stock temporairement les données de sortie

for row in tab :
    tmp_all_x.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_all_x[len(tmp_all_x)-1].append(row[1].get(input))
        else :
            tmp_all_x[len(tmp_all_x)-1].append(0)
    tmp_all_y.append(row[0])

all_x = np.array(tmp_all_x) #données en entrée du modèle
all_y = np.array(tmp_all_y) #données en sortie du modèle
print(all_x)

[[5.32620734 1.46742503 2.06089582 ... 0.         0.         0.        ]
 [0.61120412 0.60031024 0.68696527 ... 0.         0.         0.        ]
 [0.96046362 2.60134436 0.68696527 ... 0.         0.         0.        ]
 ...
 [0.         0.5336091  0.9159537  ... 0.         0.         0.        ]
 [0.         0.5336091  0.45797685 ... 0.         0.         0.        ]
 [0.         0.26680455 0.45797685 ... 0.         0.         0.        ]]


In [8]:
#normalisation v2 (la v1 est un peu mieux)
"""
for i in range(len(all_inputs)) :
    print(i)
    tmp = []
    for row in all_x :
        tmp.append(row[i])
    mean = np.mean(tmp)
    for row in all_x :
        row[i] = row[i]/mean
    
print(all_x)
"""

#enlevage des données extrèmes (faire une normalisation avant)
"""
i = 0
bad_rows = []
for row in range(len(all_x)) :
    for element in all_x[row] :
        if element > 5 :
            i +=1
            bad_rows.append(row)
            continue

all_x = np.delete(all_x,bad_rows, axis=0)
print(all_x)
"""

'\ni = 0\nbad_rows = []\nfor row in range(len(all_x)) :\n    for element in all_x[row] :\n        if element > 5 :\n            i +=1\n            bad_rows.append(row)\n            continue\n\nall_x = np.delete(all_x,bad_rows, axis=0)\nprint(all_x)\n'

In [9]:

inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)



In [10]:
history = model.fit(all_x, all_y, epochs=20, validation_split=0.1)

Epoch 1/20
44/44 [==============================] - 5s 21ms/step - loss: 1.7887 - accuracy: 0.3460 - val_loss: 1.7863 - val_accuracy: 0.3613
Epoch 2/20
44/44 [==============================] - 0s 4ms/step - loss: 1.7841 - accuracy: 0.3568 - val_loss: 1.7817 - val_accuracy: 0.3613
Epoch 3/20
44/44 [==============================] - 0s 3ms/step - loss: 1.7795 - accuracy: 0.3568 - val_loss: 1.7771 - val_accuracy: 0.3613
Epoch 4/20
44/44 [==============================] - 0s 3ms/step - loss: 1.7749 - accuracy: 0.3568 - val_loss: 1.7723 - val_accuracy: 0.3613
Epoch 5/20
44/44 [==============================] - 0s 3ms/step - loss: 1.7703 - accuracy: 0.3568 - val_loss: 1.7676 - val_accuracy: 0.3613
Epoch 6/20
44/44 [==============================] - 0s 3ms/step - loss: 1.7657 - accuracy: 0.3568 - val_loss: 1.7628 - val_accuracy: 0.3613
Epoch 7/20
44/44 [==============================] - 0s 3ms/step - loss: 1.7610 - accuracy: 0.3568 - val_loss: 1.7577 - val_accuracy: 0.3613
Epoch 8/20
44/44 [=

In [11]:
inputs = keras.Input(shape=(1,))
x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

In [12]:
for row in range(len(all_x[0])):
    tmp = []
    for column in range(len(all_x)) :
        tmp.append(all_x[column][row])
    new_x = np.array(tmp)
    print(row)
    history = model.fit(tmp, all_y, epochs=5, validation_split=0.1)

0


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'numpy.float64'>, <class 'nu